# GAN For Face Detection

## Importing

In [1]:
from torchvision.datasets import CelebA
import torchvision.transforms as ttran

## Transform

In [10]:
train_transform = ttran.Compose([ttran.PILToTensor()])
test_transform = ttran.Compose([ttran.PILToTensor()])


## Dataset

In [11]:
train_ds = CelebA('../data/', split='train', download=True, transform=train_transform)
test_ds = CelebA('../data/', split='test', download=True, transform=test_transform)

In [12]:
img, label = train_ds[0]